# Imports & globals

In [13]:
from general_postprocessing_steps import *

DEVICE = "cuda:0"

# Fetch runs and hyper params from W&B

In [14]:
# These prefixes are prefixes of the run_names in W&B overview
prefixes = ["(mdr-vae-exp 8 oct)", "(fb-vae-exp 8 oct) ", 
            "(beta-vae-exp 6 oct) ", "(inf-vae-exp 5 oct) ", 
            "(5-jan ptb-test)", "(4-jan ptb-test)", "(10-jan ptb)", 
            "(11-jan ptb)", "(19-jan ptb)", "(20-jan bmnist)", 
            "(23-jan bmnist)", "(10-feb bmnist)"]

run_df = make_run_overview_df(prefixes=prefixes)
print(len(run_df))
run_df.head()

148


,objective,l_rate,dataset,image_or_language,beta_beta,free_bits,mdr_value,l_mmd,decoder,run_name
BETA-VAE beta 0 dec: CNN.T,BETA-VAE,0,bmnist,image,0,0,0,0,basic_deconv_decoder,(10-feb bmnist) B-VAE[b=0.0] | CNN.T | Nz=10
BETA-VAE beta 0 dec: PixelCNN++,BETA-VAE,0,bmnist,image,0,0,0,0,cond_pixel_cnn_pp,(10-feb bmnist) B-VAE[b=0.0] | PixelCNN++ | Nz=10
INFO-VAE l_Rate 10 l_MMD 100 dec: CNN.T,INFO-VAE,10,bmnist,image,0,0,0,100,basic_deconv_decoder,"(23-jan bmnist) INFO-VAE[l_1_rate=10.0, l_2_mm..."
INFO-VAE l_Rate 10 l_MMD 10 dec: CNN.T,INFO-VAE,10,bmnist,image,0,0,0,10,basic_deconv_decoder,"(20-jan bmnist) INFO-VAE[l_1_rate=10.0, l_2_mm..."
INFO-VAE l_Rate 10 l_MMD 100 dec: PixelCNN++,INFO-VAE,10,bmnist,image,0,0,0,100,cond_pixel_cnn_pp,"(20-jan bmnist) INFO-VAE[l_1_rate=10.0, l_2_mm..."


# Encode & sample (conditionally & unconditionally)

In [10]:
from transformers import RobertaTokenizerFast
tokenizer = RobertaTokenizerFast.from_pretrained("roberta-base")

# 20 x 100 = 2000 (un)conditional samples
# encode all batches
encode_sample_all(run_df, device=DEVICE, include_train=True, n_sample_batches=20, force_recompute=False,
                  sample_batch_size=100, tok=tokenizer, batch_size=128)

BETA-VAE beta 0 dec: CNN.T
**** BMNIST
beta_kl: 0.00 | total_loss: 79.05 | mmd: 0.27 | elbo: -1911806.61 | distortion: 79.05 | kl_prior_post: 1911727.57 | mean_mean: -46.62 | std_across_x_mean: 98.48 | std_across_z_mean: 642.47 | mean_scale: 0.00 | std_across_x_scale: 0.00 | std_across_z_scale: 0.00 | 
Saving encodings!
Saving samples!/20
BETA-VAE beta 0 dec: CNN.T conditional sampling start: 20 x 128 = 2560 samples
only perform conditional sampling based on validation samples for now.
only perform conditional sampling based on validation samples for now.
Warning! altered code, only looping over valid in image_conditional_sample.
Saving conditonal samplings!
BETA-VAE beta 0 dec: PixelCNN++
beta_kl: 0.00 | total_loss: 60.14 | mmd: 0.53 | elbo: -219.51 | distortion: 60.14 | kl_prior_post: 159.36 | mean_mean: 0.32 | std_across_x_mean: 0.87 | std_across_z_mean: 1.44 | mean_scale: 0.00 | std_across_x_scale: 0.00 | std_across_z_scale: 0.00 | 
Saving encodings!
Saving samples!/20
BETA-VAE bet

Done this run already, continue
INFO-VAE l_Rate 1 l_MMD 1000 dec: CNN.T
Done this run already, continue
INFO-VAE l_Rate 1000 l_MMD 100 dec: PixelCNN++
Done this run already, continue


# Compute surprisals (IW LL & PPL)

In [11]:
with torch.no_grad():
    # batch size surprisal is low as the effective batch is (batch_size_surprisal * n_iw_samples))
    gather_surprisal_stats(run_df, device=DEVICE, include_train=False, force_recompute=False,
                           batch_size_surprisal=4, max_batches=125, n_iw_samples=50)

**** BMNIST
beta_kl: 0.00 | total_loss: 79.05 | mmd: 0.27 | elbo: -1911806.61 | distortion: 79.05 | kl_prior_post: 1911727.57 | mean_mean: -46.62 | std_across_x_mean: 98.48 | std_across_z_mean: 642.47 | mean_scale: 0.00 | std_across_x_scale: 0.00 | std_across_z_scale: 0.00 | 
Data
decoder_network_type basic_deconv_decoder
image_or_language image
Calculating importance weighted log likelihood (n_samples=50, batch_size=4)!
Calculating importance weighted log likelihood (n_samples=50, batch_size=4)!
Saving surprisal stats dataweighted (n_samples=50) log likelihood 125/2500
beta_kl: 0.00 | total_loss: 60.14 | mmd: 0.53 | elbo: -219.51 | distortion: 60.14 | kl_prior_post: 159.36 | mean_mean: 0.32 | std_across_x_mean: 0.87 | std_across_z_mean: 1.44 | mean_scale: 0.00 | std_across_x_scale: 0.00 | std_across_z_scale: 0.00 | 
Data
decoder_network_type cond_pixel_cnn_pp
image_or_language image
Calculating importance weighted log likelihood (n_samples=50, batch_size=4)!
Calculating importance wei

# Evaluate intrinsically (R, D, ELBO, MMD, etc.)

In [12]:
with torch.no_grad():
    evaluate(run_df, include_train=True, n_samples_mmd=5000, num_workers=3, batch_size=128, 
             force_recompute=False, device=DEVICE)

BETA-VAE beta 0 dec: CNN.T (10-feb bmnist) B-VAE[b=0.0] | CNN.T | Nz=10
**** BMNIST
beta_kl: 0.00 | total_loss: 79.05 | mmd: 0.27 | elbo: -1911806.61 | distortion: 79.05 | kl_prior_post: 1911727.57 | mean_mean: -46.62 | std_across_x_mean: 98.48 | std_across_z_mean: 642.47 | mean_scale: 0.00 | std_across_x_scale: 0.00 | std_across_z_scale: 0.00 | 
BETA-VAE beta 0 dec: PixelCNN++ (10-feb bmnist) B-VAE[b=0.0] | PixelCNN++ | Nz=10
beta_kl: 0.00 | total_loss: 60.14 | mmd: 0.53 | elbo: -219.51 | distortion: 60.14 | kl_prior_post: 159.36 | mean_mean: 0.32 | std_across_x_mean: 0.87 | std_across_z_mean: 1.44 | mean_scale: 0.00 | std_across_x_scale: 0.00 | std_across_z_scale: 0.00 | 
INFO-VAE l_Rate 10 l_MMD 100 dec: CNN.T (23-jan bmnist) INFO-VAE[l_1_rate=10.0, l_2_mmd=100.0] | CNN.T | Nz=10
Did this one already, skipping it...
INFO-VAE l_Rate 10 l_MMD 10 dec: CNN.T (20-jan bmnist) INFO-VAE[l_1_rate=10.0, l_2_mmd=10.0] | CNN.T | Nz=10
Did this one already, skipping it...
INFO-VAE l_Rate 10 l_MM